## Joining and Appending Dataframes

In [13]:
#Importing pysark and creating a session
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('joins').getOrCreate()
spark

### Working with the database which contains different csv files. All the files present in the folder are read separately using loops.

In [14]:
import os
path = "datasets-intro/uw-madison-courses/"

df_list = []

for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filename_list = filename.split(".") #separating path from .csv
        df_name = filename_list[0]
        df = spark.read.csv(path+filename,inferSchema=True,header=True)
        df.name = df_name
        df_list.append(df_name)
        exec(df_name + ' = df') #exec() function is used for the dynamic execution of Python program which can either be a string or object code
        
print("Full list of dfs:", df_list)

Full list of dfs: ['courses', 'course_offerings', 'grade_distributions', 'instructors', 'rooms', 'schedules', 'sections', 'subjects', 'subject_memberships', 'teachings']


Here is a look at some of the important variables we can use to join our tables:

 - course_offerings: uuid, course_uuid, term_code, name
 - instructors: id, name
 - schedules: uuid
 - sections: uuid, course_offering_uuid,room_uuid, schedule_uuid
 - teachings: instructor_id, section_uuid
 - courses: uuid
 - grade_distributions: course_offering_uuid,section_number
 - rooms: uuid, facility_code, room_code
 - subjects: code
 - subject_memberships: subject_code, course_offering_uuid
 
**Source:** https://www.kaggle.com/Madgrades/uw-madison-courses 

In [15]:
sections.limit(4).toPandas()

,uuid,course_offering_uuid,section_type,number,room_uuid,schedule_uuid
0,45adf63c-48c9-3659-8561-07556d2d4ddf,344b3ebe-da7e-314c-83ed-9425269695fd,FLD,1,null,f2d66a4d-0c08-3b48-abf6-649fffd7ae90
1,c6280e23-5e43-3859-893e-540d94993529,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,FLD,1,null,f2d66a4d-0c08-3b48-abf6-649fffd7ae90
2,9395dc21-15d1-3fab-8d1f-6f3fe6114c48,ea3b717c-d66b-30dc-8b37-964d9688295f,FLD,1,04368a56-c959-3e4b-8b3d-f4cc3538fea5,f2d66a4d-0c08-3b48-abf6-649fffd7ae90
3,b99e440b-39db-350a-81eb-b6eb1bd8b0bc,ea3b717c-d66b-30dc-8b37-964d9688295f,FLD,1,null,f2d66a4d-0c08-3b48-abf6-649fffd7ae90


In [16]:
rooms.limit(4).toPandas()

,uuid,facility_code,room_code
0,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,null
1,2cc50da3-ef0e-3572-a557-ca44930a0688,0032,0249
2,ebbf62b4-2ac3-356b-b0fa-7897f4446a17,0032,B101
3,ed828265-475b-31b4-b9a8-daec2a600449,0032,0549


In [19]:
course_offerings.limit(4).toPandas()

,uuid,course_uuid,term_code,name
0,344b3ebe-da7e-314c-83ed-9425269695fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1092,Cooperative Education Prog
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1082,Cooperative Education Prog
2,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog
3,075da420-5f49-3dd0-93df-13e3c152e1b1,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1114,Cooperative Education Prog


### Assigining the room numbers to each section of each course

In [18]:
step1 = rooms.join(sections, rooms.uuid == sections.room_uuid, how='left').select([rooms.uuid,rooms.facility_code, sections.course_offering_uuid,'number'])
step1.limit(4).toPandas()

,uuid,facility_code,course_offering_uuid,number
0,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,611e5499-724c-3464-82f3-1a5bf29dd96c,307
1,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,43688423-905c-3455-bf40-a58adce537f7,3
2,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,43688423-905c-3455-bf40-a58adce537f7,3
3,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,871476fd-340f-303c-a7c6-44ca6ef6b84d,311


In [22]:
step2 = step1.join(course_offerings, step1.course_offering_uuid == course_offerings.uuid, how='left').select([rooms.uuid,rooms.facility_code,'number','term_code','name'])
step2.limit(4).toPandas()

,uuid,facility_code,number,term_code,name
0,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,307,1182,International Business
1,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,3,1174,Intro Managerial Accounting
2,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,3,1174,Intro Managerial Accounting
3,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,311,1174,Business Law


### Showing the same output as above but for only facility number 0469 (facility_code)

In [24]:
step3 = step2.filter(step2.facility_code == '0469')
step3.limit(4).toPandas()

,uuid,facility_code,number,term_code,name
0,3c570ec9-1a27-3902-a16f-d1676890e63b,0469,337,1082,Introduction to Philosophy
1,3c570ec9-1a27-3902-a16f-d1676890e63b,0469,352,1082,Introduction to Philosophy
2,3c570ec9-1a27-3902-a16f-d1676890e63b,0469,308,1102,Introduction to Philosophy
3,3c570ec9-1a27-3902-a16f-d1676890e63b,0469,307,1102,Introduction to Philosophy


### Counting how many sections are offered for each subject for each facility

In [26]:
subject_memberships.limit(4).toPandas()

,subject_code,course_offering_uuid
0,220,344b3ebe-da7e-314c-83ed-9425269695fd
1,320,344b3ebe-da7e-314c-83ed-9425269695fd
2,346,344b3ebe-da7e-314c-83ed-9425269695fd
3,612,344b3ebe-da7e-314c-83ed-9425269695fd


In [27]:
subjects.limit(4).toPandas()

,code,name,abbreviation
0,908,Soil Science,SOIL SCI
1,350,ENGLISH,ENGLISH
2,351,English as a Second Language,ESL
3,230,Human Development and Family Studies,HDFS


In [28]:
step1 = subjects.join(subject_memberships, subjects.code == subject_memberships.subject_code, how='inner').select(['name','course_offering_uuid']).withColumnRenamed('name','subject_name')
step1.limit(4).toPandas()

,subject_name,course_offering_uuid
0,Chemical and Biological Engineering,344b3ebe-da7e-314c-83ed-9425269695fd
1,Electrical and Computer Engineering,344b3ebe-da7e-314c-83ed-9425269695fd
2,Engineering Mechanics and Astronautics,344b3ebe-da7e-314c-83ed-9425269695fd
3,Mechanical Engineering,344b3ebe-da7e-314c-83ed-9425269695fd


In [29]:
step2 = step1.join(sections, step1.course_offering_uuid == sections.course_offering_uuid, how='left').select(['subject_name','room_uuid'])
step2.limit(4).toPandas()

,subject_name,room_uuid
0,Communication Arts,dd6118e0-7221-3b81-9b29-aad61f0ede54
1,Communication Arts,4e9c55b8-7c02-36c3-b907-3e832ea285e7
2,Communication Arts,4948c250-c6cf-3272-a497-f90962f3ba67
3,Communication Arts,698555af-8ec3-3b85-8e67-bd7fc53aba80


In [30]:
# I added a filter to make this a little simpler
step3 = step2.join(rooms, step2.room_uuid == rooms.uuid, how='left').filter('facility_code IN("0140","0545","0469","0031")').select(['subject_name','facility_code','room_code'])
step3.limit(4).toPandas()

,subject_name,facility_code,room_code
0,Communication Arts,0140,1070
1,Communication Arts,0545,4028
2,Communication Arts,0545,4008
3,Communication Arts,0469,2241
